In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import cufflinks as cf
cf.go_offline()
import matplotlib.pyplot as plt
import datetime
import folium

In [2]:
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [3]:
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,782
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,723,723,731,738,738,743,743,743,745,745
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,25,25,25,25,26,27,27,27,27,30


In [4]:
df_deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,40,42,43,47,50,57,58,60,64,68
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,27,27,27,27,28,28,30,30,31,31
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,402,407,415,419,425,432,437,444,450,453
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,37,37,40,40,40,40,41,42,42,43
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [5]:
df_deaths_india = df_deaths[df_deaths['Country/Region']=='India']
df_deaths_india = df_deaths_india.drop(['Province/State','Country/Region','Lat','Long'],axis = 1)
df_deaths_india = df_deaths_india.transpose().reset_index()
df_deaths_india.rename(columns = {'index':'Date',131:'Deaths'}, inplace = True)
df_deaths_india['Date'] = pd.to_datetime(df_deaths_india['Date'])

df_confirmed_india = df_confirmed[df_confirmed['Country/Region']=='India']
df_confirmed_india = df_confirmed_india.drop(['Province/State','Country/Region','Lat','Long'],axis = 1)
df_confirmed_india = df_confirmed_india.transpose().reset_index()
df_confirmed_india.rename(columns = {'index':'Date',131:'Confirmed'}, inplace = True)
df_confirmed_india.drop('Date', axis = 1,inplace = True)
india_covid = pd.concat([df_confirmed_india, df_deaths_india], axis = 1)
india_covid['mortality_rate'] = india_covid['Deaths']*100/india_covid['Confirmed']

In [18]:
india_covid[40:].iplot(x = 'Date', y = ['Confirmed' , 'Deaths'], kind= 'line')

In [7]:
india_covid[40:].iplot(x = 'Date', y = 'mortality_rate', kind= 'line', color = 'green',title = 'Mortality Rate')

In [8]:
i = 0
oldx =0
def new_conf(x):
    global i, oldx
    if i == 0:
        oldx = x
    diff = x - oldx 
    oldx = x
    i = i+1
    return diff

In [9]:
india_covid['new_confirmed'] = india_covid['Confirmed'].apply(lambda x:new_conf(x))


In [10]:
india_covid['new_deaths'] = india_covid['Deaths'].apply(lambda x:new_conf(x))

In [11]:
india_covid[40:].iplot(x = 'Date', y = 'new_confirmed', kind= 'bar', xTitle='Date', title = 'new confirmed')

In [12]:
india_covid.iloc[40:].iplot(x = 'Date', y = 'new_deaths', kind= 'bar', xTitle='Date', title = 'new deaths', color = 'red')

In [13]:
df_confirmed.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,782


In [14]:
world_map = folium.Map(location=[10,0], tiles="cartodbpositron", zoom_start=2,max_zoom=6,min_zoom=2)
for i in range(0,len(df_confirmed)):
    folium.Circle(
        location=[df_confirmed.iloc[i]['Lat'], df_confirmed.iloc[i]['Long']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+df_confirmed.iloc[i]['Country/Region']+"</h5>"+
                    "<div style='text-align:center;'>"+str(np.nan_to_num(df_confirmed.iloc[i]['Province/State']))+"</div>"+
                    "<hr style='margin:10px;'>"+
                    "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        "<li>Confirmed: "+str(df_confirmed.iloc[i,-1])+"</li>"+
        "<li>Deaths:   "+str(df_deaths.iloc[i,-1])+"</li>"+
        "<li>Mortality Rate:   "+str(np.round(df_deaths.iloc[i,-1]/(df_confirmed.iloc[i,-1]+1.00001)*100,2))+"</li>"+
        "</ul>"
        ,
        radius=(int((np.log(df_confirmed.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='green',
        fill_color='yellow',
        fill=True).add_to(world_map)

world_map